<a href="https://colab.research.google.com/github/SrivenkateswaraRao/Research-Project/blob/master/Pest_Classifier_CNN1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
from os import listdir
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [7]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [8]:
import tensorflow as tf 
import numpy as np


In [9]:
import keras as ks 

Using TensorFlow backend.


# import the necessary packages


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from google.colab.patches import cv2_imshow



In [2]:
from google.colab import files
files.upload() 

Saving CreatePestImageData.py to CreatePestImageData.py


{'CreatePestImageData.py': b'# -*- coding: utf-8 -*-\r\n"""createImageDataset.ipynb\r\n\r\nAutomatically generated by Colaboratory.\r\n\r\nOriginal file is located at\r\n    https://colab.research.google.com/drive/1FXiYKCQ-jtbL_O29BQb2Xy1bBd2inM9q\r\n"""\r\n\r\nfrom os import listdir\r\nimport numpy as np\r\nimport xml.etree.ElementTree as ET\r\nimport cv2\r\nimport tensorflow as tf \r\n\r\nclass createImageDataset(object):\r\n    \r\n    def __init__(self, class_map=None):\r\n        self.image_info = []\r\n        self.trainX = []\r\n        self.trainY = []\r\n        self.testX = []\r\n        self.testY = []\r\n            \r\n    def add_image_info(self , image_array , image_path , image_id , object_id , class_name , class_id):\r\n        \r\n        image_info_ = {\'object_id\': image_id,\r\n                       \'object_class\': class_name,\r\n                       \'image_path\': image_path,\r\n                       \'image_array\': image_array,\r\n                       \

In [3]:
import CreatePestImageData as Imagedataset

In [11]:
file_list = drive.ListFile(
    {'q': "'1MzurTS9xeNSss_hYgko4ATZ5qNVn45ty' in parents"}).GetList()

for f in file_list:
    print('title: %s, id: %s' % (f['title'], f['id']))
    fname = f['title']
    print('downloading to {}'.format(fname))
    f_ = drive.CreateFile({'id': f['id']})
    f_.GetContentFile(fname)

title: LABELS.zip, id: 1dYxm-yV2sCOxiQQSvzRY6-gKtlfo0WLH
downloading to LABELS.zip
title: GREENPATROL_DB_V1.0.zip, id: 1wJw1Ci1OTUoKjiiT3lwWqwonTunGPMB1
downloading to GREENPATROL_DB_V1.0.zip
title: predefined_classes.txt, id: 1XQkqHvepaPEXxJ5XXSZmMMk9qrxTOSzJ
downloading to predefined_classes.txt


In [12]:
import CreatePestImageData as Imagedataset

In [13]:
download_Pests = drive.CreateFile({'id': '1wJw1Ci1OTUoKjiiT3lwWqwonTunGPMB1'})


In [14]:
download_Annotations = drive.CreateFile({'id': '1dYxm-yV2sCOxiQQSvzRY6-gKtlfo0WLH'})


In [ ]:
!unzip LABELS.zip -d '/Pests'


In [ ]:
!unzip GREENPATROL_DB_V1.0.zip -d '/Pests'


# Data PreProcessing and Transformation 

In [17]:
class_object = {0 : 'background' , 1 : 'egg_wf' ,
                2 : 'egg_bt', 3 : 'egg_ta', 4 : 'wf', 5 : 'bt', 6 : 'ta', 7 : 'ta_maps', 8 : 'ta_larvae', 9 : 'ta_maps+larvae', 10 : 'tomato'}

In [18]:
dataSet = Imagedataset.createImageDataset()

In [ ]:
 dataSet.prepare_dataset('/Pests/LABELS' , class_object,   test_size=10)

In [84]:
(trainY , trainX) , (testY , testX ) = (dataSet.trainY , dataSet.trainX ) , (dataSet.testY , dataSet.testX )



In [86]:
# define the model
model_with_conv = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu, input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Flatten(),    
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    #  10-node softmax layer, with each node representing a class of clothing.
    tf.keras.layers.Dense(10,  activation=tf.nn.softmax)
])
 
# compile the model
model_with_conv.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
print(model_with_conv.summary())

In [87]:
#model_with_conv.fit(x=[trainX, trainY], y=trainY, verbose=1)
model_with_conv.fit(trainX, trainY, epochs=5)
#model_with_conv.evaluate(testX,  testY, verbose=2)

Epoch 1/5
7/7 [==============================] - 0s 45ms/step - loss: 1.0616 - accuracy: 0.7282
Epoch 2/5
7/7 [==============================] - 0s 43ms/step - loss: 0.4975 - accuracy: 0.8155
Epoch 3/5
7/7 [==============================] - 0s 41ms/step - loss: 0.3748 - accuracy: 0.8689
Epoch 4/5
7/7 [==============================] - 0s 41ms/step - loss: 0.3071 - accuracy: 0.8883
Epoch 5/5
7/7 [==============================] - 0s 41ms/step - loss: 0.2659 - accuracy: 0.8738
